# Detekce obličeje pomocí termokamery

Cvičení je zaměřené na práci s [termokamerami](https://en.wikipedia.org/wiki/Thermographic_camera). Termokamera je zařízení, které snímá obraz v IR spektru, podobně jako RGB kamery, které snímají ve viditelném spektru. Výstupem termokamery je `Radiometric image`, ve kterém je v každém pixelu zakódována hodnota nasnímané teploty v proprietárním formátu. Dnešní pokročilejší software dovoluje exportovat nasnímané obrázky ve formátu `csv`, ve kterém jsou umístěné skutečné hodnoty teplot ve škále pro nás známé.


Termokamery jsou aktivně používané v průmyslu, např. pro kontrolu zahřívání elektických součástek nebo kotrolu vadností solárních panelů pomocí dronů.

![](images/solar.png)

### Import knihoven a konfigurace

In [ ]:
%run ../svz.ipynb
from collections import OrderedDict

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

Třída pro práci s popisnými charakteristikami tvarů a jednotlivé dílčí funkce. **Pozor, že tady fukce jako parametr berou obrázek a NE konturu!** 

- [`roundness(...)`](../svz.ipynb#shape_desc_funtions)
- [`aspect_ratio(...)`](../svz.ipynb#shape_desc_funtions)
- [`convexity(...)`](../svz.ipynb#shape_desc_funtions)
- [`solidity(...)`](../svz.ipynb#shape_desc_funtions)
- [`compactness(...)`](../svz.ipynb#shape_desc_funtions)
- [`extent(...)`](../svz.ipynb#shape_desc_funtions)

Další zajímavé funkce
- [`crop_by_bounding_rect(...)`](../svz.ipynb#preprocessing_functions)
- [`crop_contour(...)`](../svz.ipynb#preprocessing_functions)
- [`contour_to_image(...)`](../svz.ipynb#preprocessing_functions)

Normalizace obrázku a převedení do BGR

In [ ]:
def normalize_image(img):
    scaled = ((img - img.min()) * (1/(img.max() - img.min()) * 255)).astype('uint8')
    return cv2.cvtColor(scaled, cv2.COLOR_GRAY2BGR)

Načtení CSV souboru do obrázku

In [ ]:
def load_csv(file_path):
    return np.genfromtxt(file_path, delimiter=';')[:, :-1]

---

### Úkol
Detekujte obličej na termosnímku, použijte znalosti klasifikace pomocí tvarových chrakteristik z předchozího cvičení.

#### 1) Získejte termosnimek s obličejem
Na vašem počítači je k dispozici program `Core player`, pomocí kterého se dá získat termosnímek. Potřebujeme data s hodnotami teplot ve formátu `csv`.

Pro zjednodušení zobrazování lze použít pomocnou funkci `normalize_image`, která převede csv obrázek na BGR obrázek.

In [ ]:
file_path = '' ###
img_csv = ...(file_path) ###

plot_images(...(img_csv)) ###

#### 2) Segmentace obličeje
Vymyslete způsob, jak se dá segmentovat obličej v termosnímku. Využijte znalosti o **teplotě** lidské kůže (NEPOŽÍVEJTE normalizované šedotónové obrázky s hodnotami jasu, neboť to **nebude fungovat obecně**!). Zkuste experimentovat s parametry na více snímkách s vámi a vaším kolegou.

In [ ]:
bin_mask = ...(img_csv, ?, ?) ###
plot_images(bin_mask)

#### 3) Získejte vlastnosti segmentovaných objektů
Získejte kontury segmentovaných objektů, a odfiltrujte šum, např. pomocí plochy kontury. Dejte pozor na to, že obličej  může mít na různých snímkách různé rozměry.

U získaných kontur spočtěte popisné tvarové charakteristiky a vypište jejich hodnoty (viz [Pomocné funkce](#Pomocné-funkce)). Použijte znalosti z minulého cvičení.

In [ ]:
contours_bin, _, conts = ...(bin_mask, ?) ###
faces = mask(img_csv, contours_bin) ###
plot_images(faces)

In [ ]:
# výpis hodnot tvarových charakteristik
for contour in conts:
    print('Délka:        ', len(contour))
    print('Plocha:       ', cv2.contourArea(contour))
    
    ### další charakteristiky
    
    
    
    
    
    
    
    
    print('--------------------------------')

#### 4) Vyberte ručně významné vlastnosti a jejich hodnoty na odlišení obličeje od ostatních objektů
Vytvořte proměnné pro prahy (limity) hodnot tvarových charakteristik použitelných pro filtraci kontur. Vytvořte vhodné podmínky pro filtraci kontur.

In [ ]:
# Příklad, jak by mohly vypadat proměnné pro prahy (limity)
..._thresh = ###
..._thresh = ###
### ...

filtered_conts = []
for contour in conts:
    face_candidate = ... ### vytvoření obrázku z kontury
    
    # vhodný počet podmínek
    if ... and ...:
        filtered_conts.append(contour)
        
print('Počet nalezených obličejů je ' + str(len(filtered_conts)))

#### 5) Udělejte vizualizaci výsledné detekce
Použijte aproximace obdélníkem nebo elipsou pro demonstrační účely ([ukázka](https://docs.opencv.org/3.1.0/dd/d49/tutorial_py_contour_features.html)).

In [ ]:
demo_image = ...(img_csv) ###
for cont in filtered_conts:
    ### nakreslení obdélníku/elipsy do obrázku
    
    
plot_images(demo_image)

#### 6) Vyzkoušejte vás detekční algoritmus na referenčním obrázku

In [ ]:
filenames = ['data/1.csv', 'data/2.csv']

### celý algoritmus v jednom včetně vizualizace konečného výsledku
